<a href="https://colab.research.google.com/github/LisaJAO/UpdateExchangeRateNow/blob/master/Copy_of_sql_flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL

## 為什麼要用資料庫？
- 可以隨心所欲的做資料的存取
- 當資料龐大時，處理效率比直接對檔案IO快上許多
- 方便跟各種服務系統串接

## 資料庫種類
- 關聯式資料庫：資料欄位須先定義清楚、穩定
- 非關聯式資料庫：資料擁有彈性結構、速度較快

## 常見的關聯式資料庫種類
- MySQL：免費、開源性質、常應用於Web，較適合中小型企業事務
- PostgreSQL：免費、開源、以商業應用導向為主
- MSSQL：Microsoft SQL Server，only on Windows，花錢就能買的完整方案。
- Oracle：公部門常用的資料庫系統，安全、效能、功能都極為齊全。
- SQLite：基於文件的資料庫，不需要架設伺服器，就能處理各種類數據。

## 常見的非關聯式資料庫種類
- MongoDB
- Couchbase
- Cassandra
- Redis
- memcache
- HBase
- Neo4J
  
![](http://image.slidesharecdn.com/mongodbrdbmsmigrationwebinar-140202060406-phpapp02/95/migrating-from-relational-databases-to-mongodb-10-638.jpg?cb=1391321199)

## SQLite

``` python
import sqlite3
conn = sqlite3.connect('資料庫名稱.db') 
sql = "......"
cursor = conn.execute(sql_ins)
cursor = conn.commit() ###  會修改到資料的必須要多寫此行
conn.close()
```

### 建立資料庫/資料表
- 建立資料庫(Database/DB)：`資料庫.db`
- 建立資料表(Table)：

SQLite 型別
- INTEGER：整數
- REAL：浮點數
- TEXT：文字
- BLOB：原始資料
  
``` mysql
CREATE TABLE `資料表` (
	`欄位1`	型別,
	`欄位2`	型別,
	`.....
)
```

In [0]:
import sqlite3
conn = sqlite3.connect('stocks.db') # 如果資料庫不存在，會自動幫你建立
sql_create_table = """
CREATE TABLE `stock_date` (
	`stock_id`	INTEGER,
	`date`	TEXT,
	`open`	REAL,
	`high`	REAL,
	`low`	REAL,
	`close`	REAL,
	`volume`	INTEGER
)
"""
cursor = conn.execute(sql_create_table)
conn.close()

##### [練習]
- 寫出 建立資料表 stock_list 的 sqlite 語法
    - 股票代號 stock_id
    - 股票名稱 name
    - 想想看以上欄位要用什麼型別

## CRUD

### Create 新增資料
``` mysql
INSERT INTO `資料表` (`欄位1`, `欄位2`, ...)
VALUES (值1, 值2, )
```

In [0]:
import sqlite3
conn = sqlite3.connect('stocks.db')
sql_ins = """
    INSERT INTO `stock_date` (`stock_id`, `date`, `open`, `high`, `low`, `close`, `volume`)  
    VALUES ( 2330 ,'2020-03-03', 318.5, 320, 316, 317.5, 55169 )
"""
cursor = conn.execute(sql_ins)
cursor = conn.commit()
conn.close()

#### [練習]
將以下資料加入資料庫
- 要加上股票代號
- 日期要轉格式
- 型別轉換(好的習慣)


In [0]:
datas_2330 = [
    ['2020/03/02', '308', '317', '308', '311', '86373'],
    ['2020/03/03', '318.5', '320', '316', '317.5', '55169'],
    ['2020/03/04', '322', '322', '317', '320.5', '44745'],
    ['2020/03/05', '325', '326', '323', '323', '38224'],
    ['2020/03/06', '320', '320.5', '315', '315', '52808']
]
datas_2317 = [
    ['2020/03/02', '78.2', '80.3', '78.1', '79.2', '54992'],
    ['2020/03/03', '81.1', '82', '80.6', '81', '34822'],
    ['2020/03/04', '81.5', '81.9', '81.1', '81.7', '29908'],
    ['2020/03/05', '83.3', '83.6', '82.6', '82.7', '36950'],
    ['2020/03/06', '81.6', '81.7', '80.7', '80.8', '38713']
]

### Read 讀取資料 
``` mysql
SELECT `欄位1`, `欄位2`, ...
FROM `資料表`
WHERE 條件1 and/or 條件2 ....
ORDER BY 排序方式1, 排序方式2
LIMIT 筆數限制
```
- 顯示欄位：
    - 全選所有資料表內的欄位 `SELECT * `
    - 可自訂新欄位(運算符號和Python相同) `SELECT 欄位1, (欄位1-欄位2)*100/欄位2`
    - 可自訂順序(不需要跟資料表順序一致)
- 條件：
    - 數值比較符號大致上和Python相同，
        - 比較兩邊相等：`=`
        - 沒有整除、次方運算子(要用[函式](https://www.w3schools.com/sql/sql_ref_mysql.asp))
- 排序方式：
    - DESC 倒序(大->小)排序
    - ASC 正序(小->大)排序(可省略不寫)

In [0]:
import sqlite3
conn = sqlite3.connect('stocks.db')
sql = """
    SELECT `stock_id`,`date`,`open`,`close`,`high`,`low`,`volume`, `volume`*1000
    FROM `stock_date`
    WHERE `date` = '2020-03-03' or `volume` >= 40000
    ORDER BY `stock_id` DESC, `date`
"""
cursor = conn.execute(sql)
for row in cursor.fetchall():
    print(row)
conn.close()

#### [練習]
列出 台積電(2330) 2020-03-04 之後的 每日資訊
- 日期
- 開盤、收盤、最高、最低
- 均價 CDP：(最高+最低+2*收盤) / 4
- 列出擁有最大「最高價」的兩筆資料

### Update 更新資料
``` mysql
UPDATE `資料表`
SET `欄位1`=值1, `欄位2`=值2, ...
WHERE 條件1 and/or 條件2 ....
```

In [0]:
import sqlite3
conn = sqlite3.connect('stocks.db')
sql_upd = """
    UPDATE `stock_date`
    SET `open`=`open`+1000, `close`=900
    WHERE `stock_id` = 2330 and `date` = '2020-03-03'
"""
cursor = conn.execute(sql_upd)
cursor = conn.commit()
conn.close()

In [0]:
import sqlite3
conn = sqlite3.connect('stocks.db')
sql = """
    SELECT * 
    FROM `stock_date`
    WHERE `stock_id` = 2330 and `date` = '2020-03-03'
"""
cursor = conn.execute(sql)
for row in cursor.fetchall():
    print(row)
conn.close()

### Delete 刪除資料
``` mysql
DELETE 
FROM `資料表`
WHERE 條件1 and/or 條件2 ....
```

In [0]:
import sqlite3
conn = sqlite3.connect('stocks.db')
sql_upd = """
    DELETE 
    FROM `stock_date`
    WHERE `stock_id` = 2330 and `date` = '2020-03-03'
"""
cursor = conn.execute(sql_upd)
cursor = conn.commit()
conn.close()

In [0]:
import sqlite3
conn = sqlite3.connect('stocks.db')
sql = """
    SELECT * 
    FROM `stock_date`
"""
cursor = conn.execute(sql)
for row in cursor.fetchall():
    print(row)
conn.close()

### 處理重複資料
- 新增資料的時候先檢查資料是否存在：`SELECT`
- 設計資料庫：
    - 新增一欄位讓所有資料各擁有一唯一代碼，
        - 刪除時留下最小或最大的代碼：先用`SELECT`，再用`DELETE`
    - 建立唯一值組合：`UNIQUE INDEX`
        - 要先思考哪些欄位組合起來是唯一值

In [0]:
import sqlite3
conn = sqlite3.connect('stocks.db')
sql_create_ui = """
    CREATE UNIQUE INDEX `id_date` ON `stock_date`(`stock_id`, `date`)
"""
cursor = conn.execute(sql_create_ui)
cursor = conn.commit()
conn.close()

In [0]:
import sqlite3
conn = sqlite3.connect('stocks.db')
sql_ins = """
    INSERT INTO `stock_date` (`stock_id`, `date`, `open`, `high`, `low`, `close`, `volume`)  
    VALUES ( 2330 ,'2020-03-03', 318.5, 320, 316, 317.5, 55169 )
"""
cursor = conn.execute(sql_ins)
cursor = conn.commit()
sql_ins2 = """
    INSERT INTO `stock_date` (`stock_id`, `date`, `open`, `high`, `low`, `close`, `volume`)  
    VALUES ( 2330 ,'2020-03-03', 918.5, 320, 316, 317.5, 55169 )
"""
cursor = conn.execute(sql_ins2)
cursor = conn.commit()
conn.close()

In [0]:
import sqlite3
conn = sqlite3.connect('stocks.db')
sql = """
    SELECT * 
    FROM `stock_date`
"""
cursor = conn.execute(sql)
for row in cursor.fetchall():
    print(row)
conn.close()

## MySQL

``` python
import pymysql
conn = pymysql.connect(host,account,passwd,dateabase)
cursor = conn.cursor() 
sql = "......"
cursor.execute(sql)
cursor.commit()
conn.close()
```

### 環境設定

In [0]:
!pip install PyMySQL

### [IMDB 資料庫](https://relational.fit.cvut.cz/dataset/IMDb)
![](https://relational.fit.cvut.cz/assets/img/datasets-generated/imdb_ijs.svg)


## INNER JOIN
- 一對多關係
    - 電影有很多演員
- 資料單一地方管理
    - `stock_list`：股票代碼、股票名稱
    - `stock_date`：股票代碼、日期、各交易原始資訊
    - 若股票名稱改名，改 `stock_list` 就好

``` mysql
SELECT `資料表n`.`欄位1`, `資料表n`.`欄位2`, ...
FROM `資料表1`
INNER JOIN `資料表2` ON 關聯條件1 and/or 條件2
WHERE 條件1 and/or 條件2 ....
ORDER BY 排序方式1, 排序方式2
LIMIT 筆數限制
```

In [0]:
import pymysql
conn = pymysql.connect("relational.fit.cvut.cz","guest","relational","imdb_ijs")
cursor = conn.cursor() 
sql = """
    SELECT 
        `movies`.`id`, 
        `movies`.`name`, 
        `movies`.`year`,
        `actors`.`first_name`,
        `actors`.`last_name`
    FROM `movies` 
    INNER JOIN `roles` ON `movies`.`id` = `roles`.`movie_id`
    INNER JOIN `actors` ON `roles`.`actor_id` = `actors`.`id`
    WHERE `movies`.`year`>2000
    ORDER BY `movies`.`year` DESC , `movies`.`id` 
    LIMIT 5
"""
cursor.execute(sql)
names = tuple(description[0] for description in cursor.description)
print(names)
for row in cursor.fetchall():
    print(row)
conn.close()

## 學習資源
- [w3school](https://www.w3schools.com/sql/default.asp)
- [深入淺出 SQL](https://www.tenlong.com.tw/products/9789866840166)

# Flask

## 為什麼要學架網站？
- 讓任何人不限裝置/平台/系統，只要能上網，都能很方便且易使用
- 網路爬蟲抓的就是網站，所以理解網站怎麼架設，對於增加爬蟲經驗會很有幫助

## 為什麼要用 Flask ？
- 容易上手
- 輕量化

## 環境設定

In [0]:
!pip install flask-ngrok

## 基本 route

In [0]:
from flask_ngrok import run_with_ngrok
from flask import Flask
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

@app.route("/")
def home():
    resp = "Welcome to Stock Board <br> <a href='/stock'>進入股票頁</a>"
    return resp

@app.route("/stock")
def stock():
    stock_id = "2330"
   
    resp = """
        <h1>股票代碼: {}</h1>
        <a href='/'>回首頁</a>
        <table>
            <thead>
                <tr>
                    <th>日期</th>
                    <th>開盤價</th>
                    <th>最高價</th>
                    <th>最低價</th>
                    <th>收盤價</th>
                    <th>成交量</th>
                </tr>
            </thead>
            <tbody>
                <tr>
                    <td>2020-03-02</td>
                    <td>308</td>
                    <td>317</td>
                    <td>308</td>
                    <td>311</td>
                    <td>86373</td>
                </tr>
                <tr>
                    <td>2020-03-03</td>
                    <td>318.5</td>
                    <td>320</td>
                    <td>316</td>
                    <td>317.5</td>
                    <td>55169</td>
                </tr>
                <tr>
                    <td>2020-03-04</td>
                    <td>322</td>
                    <td>322</td>
                    <td>317</td>
                    <td>320.5</td>
                    <td>44745</td>
                </tr>
            </tbody>
        </table>
    """.format(stock_id)
    return resp
app.run()

In [0]:
from flask_ngrok import run_with_ngrok
from flask import Flask
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
    resp = "Welcome to Stock Board <br> <a href='/stock'>進入股票頁</a>"
    return resp

@app.route("/stock")
def stock():
    stock_id = "2330"
    datas_2330 = [
        ['2020-03-02', 308, 317, 308, 311, 86373],
        ['2020-03-03', 318.5, 320, 316, 317.5, 55169],
        ['2020-03-04', 322, 322, 317, 320.5, 44745]
    ]
    html_data = ""
    for row in datas_2330:
        d, o, h, l, c, v = row
        html_row = """
            <tr>
                <td>{}</td>
                <td>{}</td>
                <td>{}</td>
                <td>{}</td>
                <td>{}</td>
                <td>{}</td>
            </tr>""".format(d, o, h, l, c, v)
        html_data+=html_row
    resp = """
        <h1>股票代碼: {}</h1>
        <a href='/'>回首頁</a>
        <table>
            <thead>
                <tr>
                    <th>日期</th>
                    <th>開盤價</th>
                    <th>最高價</th>
                    <th>最低價</th>
                    <th>收盤價</th>
                    <th>成交量</th>
                </tr>
            </thead>
            <tbody>
                {}
            </tbody>
        </table>
    """.format(stock_id,html_data)
    return resp
app.run()

#### [練習]
寫出股票列表頁：
- 網址 stock_list
- 有三列兩欄(股票代號、股票名)
    - 2330 台積電
    - 2317 鴻海
    - 2412 中華電


## Template
- 新建資料夾 `templates`
- 在 `templates` 資料夾中，新增檔案 `stock.html`

templates/stock.html
``` html
<h1>股票代碼: {{ stock_id }}</h1>
<a href='/'>回首頁</a>
<table>
    <thead>
        <tr>
            <th>日期</th>
            <th>開盤價</th>
            <th>最高價</th>
            <th>最低價</th>
            <th>收盤價</th>
            <th>成交量</th>
        </tr>
    </thead>
    <tbody>
        {% for row in datas %}
        <tr>
            <td>{{ row[0] }}</td>
            <td>{{ row[1] }}</td>
            <td>{{ row[2] }}</td>
            <td>{{ row[3] }}</td>
            <td>{{ row[4] }}</td>
            <td>{{ row[5] }}</td>
        </tr>
        {% endfor %}
    </tbody>
</table>
```

In [0]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
    resp = "Welcome to Stock Board <br> <a href='/stock'>進入股票頁</a>"
    return resp

@app.route("/stock")
def stock():
    stock_id = "2330"
    datas = [
        ['2020-03-02', 308, 317, 308, 311, 86373],
        ['2020-03-03', 318.5, 320, 316, 317.5, 55169],
        ['2020-03-04', 322, 322, 317, 320.5, 44745]
    ]
    return render_template("stock.html", stock_id=stock_id, datas = datas)
app.run()

#### [練習]
將首頁也改成 Template (檔案名稱：index.html)

## Parameter

In [0]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template
from flask import request
app = Flask(__name__)
run_with_ngrok(app)

def get_stock_datas(stock_id):
    datas = [
        ['2020-03-02', 308, 317, 308, 311, 86373],
        ['2020-03-03', 318.5, 320, 316, 317.5, 55169],
        ['2020-03-04', 322, 322, 317, 320.5, 44745]
    ]
    return datas

@app.route("/")
def home():
    resp = """Welcome to Stock Board 
    <br> 
    <a href='/stock/2330'>進入2330股票頁(路徑)</a> 
    <br> 
    <a href='/stock?stock_id=2330'>進入2330股票頁(參數)</a>
    """
    return resp

@app.route("/stock/<stock_id>")
def get_stock_path(stock_id):
    return render_template("stock.html", stock_id=stock_id, datas = get_stock_datas(stock_id))

@app.route("/stock")
def get_stock_param():
    stock_id = request.args.get('stock_id')
    return render_template("stock.html", stock_id=stock_id, datas = get_stock_datas(stock_id))

app.run()

#### [練習]
- 增加一個參數 date_start
    - 如果是走 get_stock_path ， url 是 `/stock/2330/XXX`
    - 如果是走 get_stock_param ， url 是 `/stock?stock_id=2330&date_start=XXX`


## SQL

In [0]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template
from flask import request
import sqlite3
app = Flask(__name__)
run_with_ngrok(app)

def get_stock_datas(stock_id):
    conn = sqlite3.connect('stocks.db')
    sql = """
        SELECT `date`, `open`, `high`, `low`, `close`, `volume`
        FROM `stock_date`
        WHERE `stock_id` = {}
        ORDER BY `date`
    """.format(stock_id)
    cursor = conn.execute(sql)
    datas = cursor.fetchall()
    return datas

@app.route("/")
def home():
    resp = """Welcome to Stock Board 
    <br> 
    <a href='/stock/2330'>進入2330股票頁(路徑)</a> 
    <br> 
    <a href='/stock?stock_id=2330'>進入2330股票頁(參數)</a>
    """
    return resp

@app.route("/stock/<stock_id>")
def get_stock_path(stock_id):
    return render_template("stock.html", stock_id=stock_id, datas = get_stock_datas(stock_id))

@app.route("/stock")
def get_stock_param():
    stock_id = request.args.get('stock_id')
    return render_template("stock.html", stock_id=stock_id, datas = get_stock_datas(stock_id))

app.run()

#### [練習]
延續上一個練習，參數 date_start 是為 sql 的條件：  
搜尋此日期(包含) 後的結果
- 注意日期是文字格式


## [Bootstrap](https://getbootstrap.com/)
- 包含 HTML、CSS及JavaScript的前端框架，有許多已經設定好的元素，方便開發

## [DataTables](https://datatables.net/)
- jQuery 套件，讓表格可以有更多功能操作
    - jQuery：JavaScript函式庫，簡化 HTML 與 JavaScript 之間的操作
- 此套件會把所有結果按照套件設定重新排序，亦即不會依照原先資料的排序

![](https://simpliv.files.wordpress.com/2018/04/learn-the-foundations-of-html-css-javascript-from-experts12.gif?w=640)

In [0]:
!pip install flask-bootstrap

### [Templates下載](https://github.com/MarsW/slides/tree/master/codes/flask/stock/templates)
- b_index.html
- b_stock.html
- bd_stock.html

In [0]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template
from flask import request
from flask_bootstrap import Bootstrap
app = Flask(__name__)
bootstrap = Bootstrap(app)
run_with_ngrok(app)

def get_stock_datas(stock_id):
    conn = sqlite3.connect('stocks.db')
    sql = """
        SELECT `date`, `open`, `high`, `low`, `close`, `volume`
        FROM `stock_date`
        WHERE `stock_id` = {}
        ORDER BY `date`
    """.format(stock_id)
    cursor = conn.execute(sql)
    datas = cursor.fetchall()
    return datas

@app.route("/")
def home():
    return render_template("b_index.html")

@app.route("/stock/<stock_id>")
def get_stock_path(stock_id):
    return render_template("b_stock.html", stock_id=stock_id, datas = get_stock_datas(stock_id))

@app.route("/stock")
def get_stock_param():
    stock_id = request.args.get('stock_id')
    return render_template("bd_stock.html", stock_id=stock_id, datas = get_stock_datas(stock_id))

app.run()

## 學習資源
- [Flask Web 開發實戰](https://tw.pyladies.com/events/topic.html?id=35)
    - [Flask Web 開發:基於 Python 的 Web 應用開發實戰](https://www.tenlong.com.tw/products/9787115373991)
- 學習地圖
    - [前端](https://softnshare.com/webfrontendprogrammer/)
    - [後端](https://softnshare.com/backenddeveloper/)
        - VPS 主機：Linode, DigitalOcean, GCP, AWS, Azure
            - [鳥哥的 Linux 私房菜](http://linux.vbird.org/)
